In [1]:
# Import Libraries 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Import the file 
File = os.path.join('..', 'data', 'raw', 'ai_job_dataset.csv')
df = pd.read_csv(File) 

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().any().sum()

In [ ]:
df[["salary_currency","salary_usd"]]

In [3]:
# Step 1: Replace job_id in df with integers
job_id_map = {job_id: i + 1 for i, job_id in enumerate(df['job_id'].unique())}
df['job_id'] = df['job_id'].map(job_id_map)

In [4]:
from collections import Counter

all_skills = []

for skill_list in df["required_skills"]:
    skills = [s.strip() for s in skill_list.split(",")]
    all_skills.extend(skills)

skill_counts = Counter(all_skills)

skills_set = set(all_skills)

# To display the frequencies
dict1 = dict()
for skill, count in skill_counts.items():
    print(f"{skill}: {count}")
    dict1[skill] = count

Tableau: 2341
PyTorch: 2777
Kubernetes: 3009
Linux: 2705
NLP: 2145
Deep Learning: 2189
AWS: 2018
Mathematics: 1943
Python: 4450
Docker: 1862
Java: 2578
Hadoop: 2419
Scala: 2794
SQL: 3407
MLOps: 2164
Data Visualization: 2270
R: 2311
Computer Vision: 2284
Azure: 2144
Git: 2631
GCP: 2442
TensorFlow: 3022
Statistics: 1833
Spark: 2155


In [5]:
skills_df = pd.DataFrame({"skill_id": [i + 1 for i in range(len(skills_set))], "skill": list(skills_set)})


In [6]:
skills_df

,skill_id,skill
0,1,TensorFlow
1,2,Hadoop
2,3,Computer Vision
3,4,Data Visualization
4,5,MLOps
5,6,Azure
6,7,Python
7,8,PyTorch
8,9,Docker
9,10,Java


In [7]:
list(df.columns).index("required_skills")

10

In [8]:
jobs2skills_df = pd.DataFrame(columns=['job_id']+list(skills_set))
jobs2skills_df["job_id"] = df['job_id']
jobs2skills_df.head()

for row in range(jobs2skills_df.shape[0]):
    print(f"Checking row: {row+1}")
    for col in range(1, jobs2skills_df.shape[1]):
        if jobs2skills_df.columns[col] in df.iloc[row, 10]:
            jobs2skills_df.iloc[row, col] = 1 
        else:
            jobs2skills_df.iloc[row, col] = 0

jobs2skills_df.head()
    
        #melted_df = jobs2skills_df.melt(id_vars=["job_id"])
#melted_df[ melted_df['job_id'] == "AI00001" ]

Checking row: 1
Checking row: 2
Checking row: 3
Checking row: 4
Checking row: 5
Checking row: 6
Checking row: 7
Checking row: 8
Checking row: 9
Checking row: 10
Checking row: 11
Checking row: 12
Checking row: 13
Checking row: 14
Checking row: 15
Checking row: 16
Checking row: 17
Checking row: 18
Checking row: 19
Checking row: 20
Checking row: 21
Checking row: 22
Checking row: 23
Checking row: 24
Checking row: 25
Checking row: 26
Checking row: 27
Checking row: 28
Checking row: 29
Checking row: 30
Checking row: 31
Checking row: 32
Checking row: 33
Checking row: 34
Checking row: 35
Checking row: 36
Checking row: 37
Checking row: 38
Checking row: 39
Checking row: 40
Checking row: 41
Checking row: 42
Checking row: 43
Checking row: 44
Checking row: 45
Checking row: 46
Checking row: 47
Checking row: 48
Checking row: 49
Checking row: 50
Checking row: 51
Checking row: 52
Checking row: 53
Checking row: 54
Checking row: 55
Checking row: 56
Checking row: 57
Checking row: 58
Checking row: 59
Checki

,job_id,TensorFlow,Hadoop,Computer Vision,Data Visualization,MLOps,Azure,Python,PyTorch,Docker,...,Kubernetes,Deep Learning,Spark,R,Tableau,Linux,GCP,Mathematics,SQL,Git
0,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,1,0,0,0,0
1,2,0,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,1,0,0
2,3,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,5,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [9]:
melted_df = jobs2skills_df.melt(id_vars=["job_id"], var_name="skill")

In [10]:
melted_df

,job_id,skill,value
0,1,TensorFlow,0
1,2,TensorFlow,0
2,3,TensorFlow,0
3,4,TensorFlow,0
4,5,TensorFlow,0
...,...,...,...
359995,14996,Git,0
359996,14997,Git,0
359997,14998,Git,0
359998,14999,Git,0


In [11]:
# Step 3: Filter rows where value == 1
melted_df = melted_df[melted_df['value'] == 1]

# Step 4: Merge with skills_df to get skill_id
bridge_table = melted_df.merge(skills_df, on='skill', how='left')

# Step 5: Select and rename required columns for SQL import
bridge_table = bridge_table[['job_id', 'skill_id']].rename(columns={'skill_id': 'skill_id', 'job_id': 'job_id'})

# Step 6 (optional): Reset index for cleanliness
bridge_table.reset_index(drop=True, inplace=True)


In [12]:
bridge_table[bridge_table['job_id'] == 4]

,job_id,skill_id
14304,4,7
30133,4,13
46766,4,20
53855,4,23


In [40]:
df

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,1,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,2,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,3,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,4,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,5,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14996,Robotics Engineer,38604,USD,EN,FL,Finland,S,Finland,50,"Java, Kubernetes, Azure",Bachelor,1,Energy,2025-02-06,2025-03-25,1635,7.9,Advanced Robotics
14996,14997,Machine Learning Researcher,57811,GBP,EN,CT,United Kingdom,M,United Kingdom,0,"Mathematics, Docker, SQL, Deep Learning",Master,0,Government,2024-10-16,2024-10-30,1624,8.2,Smart Analytics
14997,14998,NLP Engineer,189490,USD,EX,CT,South Korea,L,South Korea,50,"Scala, Spark, NLP",Associate,17,Manufacturing,2024-03-19,2024-05-02,1336,7.4,AI Innovations
14998,14999,Head of AI,79461,EUR,EN,FT,Netherlands,M,Netherlands,0,"Java, Computer Vision, Python, TensorFlow",PhD,1,Real Estate,2024-03-22,2024-04-23,1935,5.6,Smart Analytics


In [70]:
skills_df

,skill_id,skill
0,0,Linux
1,1,Git
2,2,AWS
3,3,Java
4,4,NLP
5,5,Scala
6,6,MLOps
7,7,Hadoop
8,8,Data Visualization
9,9,Computer Vision


In [41]:
df.columns

Index(['job_id', 'job_title', 'salary_usd', 'salary_currency',
       'experience_level', 'employment_type', 'company_location',
       'company_size', 'employee_residence', 'remote_ratio', 'required_skills',
       'education_required', 'years_experience', 'industry', 'posting_date',
       'application_deadline', 'job_description_length', 'benefits_score',
       'company_name'],
      dtype='object')

In [105]:
df.columns

Index(['job_id_x', 'job_title', 'salary_usd', 'salary_currency',
       'experience_level', 'employment_type', 'company_location',
       'company_size', 'employee_residence', 'remote_ratio', 'required_skills',
       'education_required', 'years_experience', 'industry', 'posting_date',
       'application_deadline', 'job_description_length', 'benefits_score',
       'company_name', 'industry_id_x', 'company_id_x', 'job_id_y',
       'required_skills_id', 'industry_id_y', 'company_id_y', 'company_id'],
      dtype='object')

In [96]:
skills_df

,skill_id,skill
0,1,Linux
1,2,Git
2,3,AWS
3,4,Java
4,5,NLP
5,6,Scala
6,7,MLOps
7,8,Hadoop
8,9,Data Visualization
9,10,Computer Vision


In [53]:
# Step 1: Filter melted_df for rows where value == 1
filtered_df = melted_df[melted_df['value'] == 1].copy()

# Step 2: Clean skill columns for safe merge
filtered_df['skill'] = filtered_df['skill'].str.strip()
skills_df['skill'] = skills_df['skill'].str.strip()

# Step 3: Merge filtered_df with skills_df on 'skill' to get 'skill_id'
bridge_table = filtered_df.merge(skills_df[['skill', 'skill_id']], on='skill', how='left')

# Step 4: Select only 'job_id' and 'skill_id'
bridge_table = bridge_table[['job_id', 'skill_id']]

# Step 5: Reset index for cleanliness
bridge_table.reset_index(drop=True, inplace=True)

# Step 6: Export to CSV
bridge_table.to_csv('skills_to_job.csv', index=False)


In [92]:
bridge_table

,job_id,skill_id
0,1,1
1,4,1
2,6,1
3,10,1
4,16,1
...,...,...
59888,14980,24
59889,14987,24
59890,14988,24
59891,14997,24


In [77]:
bridge_table


,job_id,skill_id


In [94]:
job

,job_id,job_title,salary_usd,education_required,employment_type,company_id,remote_ratio,required_skills_id,salary_currency,posting_date,benefits_score,experience_level,years_experience
0,1,AI Research Scientist,90376,Bachelor,CT,1,50,,USD,2024-10-18,5.9,SE,9
1,1,AI Research Scientist,90376,Bachelor,CT,11,50,,USD,2024-10-18,5.9,SE,9
2,1,AI Research Scientist,90376,Bachelor,CT,35,50,,USD,2024-10-18,5.9,SE,9
3,1,AI Research Scientist,90376,Bachelor,CT,63,50,,USD,2024-10-18,5.9,SE,9
4,1,AI Research Scientist,90376,Bachelor,CT,88,50,,USD,2024-10-18,5.9,SE,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8725660,15000,Computer Vision Engineer,56481,PhD,PT,9205,50,,USD,2024-07-18,7.6,MI,2
8725661,15000,Computer Vision Engineer,56481,PhD,PT,9257,50,,USD,2024-07-18,7.6,MI,2
8725662,15000,Computer Vision Engineer,56481,PhD,PT,9277,50,,USD,2024-07-18,7.6,MI,2
8725663,15000,Computer Vision Engineer,56481,PhD,PT,9280,50,,USD,2024-07-18,7.6,MI,2
